# Slicer Tutorial Brain MRI

This tutorial shows you how to use the functions from 'functions_for_slicer.py'. Before you begin, please ensure you have all the required modules downloaded from the requirements.txt file.

In [3]:
import sys
import os

# Add slicer_functions/ to the Python path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..', 'slicer_functions')))

from basic_functions import *

In [4]:
# Load in the DICOM directory for the brain resection patient into slicer
current_dir = os.path.dirname(os.path.abspath('__file__'))
brain_resection_dicom_path = os.path.join(current_dir, '..', 'example_patients', 'Brain_resection')


load_DICOM(brain_resection_dicom_path)

['vtkMRMLScalarVolumeNode1',
 'vtkMRMLGridTransformNode1',
 'vtkMRMLScalarVolumeNode2',
 'vtkMRMLCommandLineModuleNode1',
 'vtkMRMLLabelMapVolumeNode1',
 'vtkMRMLSegmentationNode1']